<a href="https://colab.research.google.com/github/MrCoderXYZ/High-School-Final-Projects/blob/main/FINAL_QR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install
!pip install qrcode[pil]
!pip install pillow
!pip install shutil
!pip install qrcode[pil]
!pip install pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.1 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil


In [ ]:
import csv
import qrcode
from pathlib import Path
from PIL import Image, ImageDraw
from google.colab import files
import shutil

# Function to generate QR code with a circular logo and color variations
def generate_qr_with_logo(data, filename, output_dir, logo_image):
    # Create a QR Code object
    qr = qrcode.QRCode(
        version=10,  # Size of the QR code (1 is the smallest, 40 is the largest)
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,  # Increase the size of each module for higher resolution
        border=0,  # Set border size to 0 to remove the border completely
    )
    qr.add_data(data)
    qr.make(fit=True)

    # Get the QR code matrix (list of lists with True/False values for each module)
    matrix = qr.get_matrix()
    size = len(matrix)  # The size of the QR code matrix (n x n)

    # Increase padding size to create space around the QR code
    padding = 300  # Padding size

    # Create a base image with transparent background
    image_size = (size * 15 + padding * 2, size * 15 + padding * 2)  # Total size with padding
    qr_img = Image.new("RGBA", image_size, (255, 255, 255, 0))  # Transparent background
    draw = ImageDraw.Draw(qr_img)

    # Predefined color list for the QR code
    color_list = ["#21C8FF", "#FF6206"]

    # Function to select a color for each module
    def get_fixed_color(index):
        return color_list[index % len(color_list)]

    # Draw the QR code on the transparent image with colors
    for row in range(size):
        for col in range(size):
            if matrix[row][col]:
                x = col * 15 + padding + 7  # Center of the circle
                y = row * 15 + padding + 7  # Center of the circle
                radius = 7  # Radius of the circle
                qr_color = get_fixed_color((row + col) % len(color_list))  # Color based on position
                draw.ellipse([x - radius, y - radius, x + radius, y + radius], fill=qr_color)

    # Resize and add the logo as before
    logo = Image.open(logo_image).convert("RGBA")
    logo_size = int(min(image_size) * 0.15)  # Resize the logo to be 15% of the image size
    logo = logo.resize((logo_size, logo_size), Image.Resampling.LANCZOS)

    # Create a circular mask for the logo
    logo_mask = Image.new("L", (logo_size, logo_size), 0)
    logo_mask_draw = ImageDraw.Draw(logo_mask)
    logo_mask_draw.ellipse([0, 0, logo_size, logo_size], fill=255)

    logo = logo.convert("RGBA")
    logo_mask = logo_mask.convert("L")

    logo_x = (image_size[0] - logo_size) // 2
    logo_y = (image_size[1] - logo_size) // 2

    # Paste the logo onto the QR code image
    qr_img.paste(logo, (logo_x, logo_y), logo_mask)

    # Save the final image
    output_path = output_dir / f"{filename}.png"  # Use the filename from CSV
    qr_img.save(output_path, "PNG")

    return output_path

# Function to read URLs and filenames from a CSV file and generate QR codes
def generate_qrs_from_csv(csv_file, logo_image):
    save_path = Path("./generated_qrs")  # Directory to save the QR codes
    save_path.mkdir(parents=True, exist_ok=True)  # Create directory if it doesn't exist

    # Ensure the file is a CSV
    if not csv_file.endswith('.csv'):
        print(f"Error: {csv_file} is not a CSV file. Please upload a valid CSV file.")
        return

    try:
        with open(csv_file, 'r', newline='', encoding='utf-8-sig') as file:
            csv_reader = csv.DictReader(file)
            for row in csv_reader:
                url = row['Link']  # Adjust column name based on your CSV structure
                filename = row['filename']  # Adjust column name based on your CSV structure
                output_path = generate_qr_with_logo(url, filename, save_path, logo_image)
                print(f"QR Code generated for {url} as {output_path}")
    except UnicodeDecodeError as e:
        print(f"Error reading CSV file. Details: {e}")
        return

    # Compress the QR codes into a ZIP file for download
    zip_filename = "generated_qrs.zip"
    shutil.make_archive("generated_qrs", 'zip', save_path)
    print(f"All QR codes have been saved and compressed into {zip_filename}.")
    return zip_filename

# Main entry point of the script
if __name__ == "__main__":
    print("Please upload your CSV file and logo.")

    # Upload CSV and logo files
    uploaded = files.upload()  # Prompt user to upload the files
    csv_file = None
    logo_file = None

    for filename in uploaded.keys():
        if filename.lower().endswith('.csv'):
            csv_file = filename
        else:
            logo_file = filename

    if csv_file and logo_file:
        # Use the uploaded PNG directly
        png_logo_file = logo_file

        # Generate QR codes and return a zip file
        zip_file = generate_qrs_from_csv(csv_file, png_logo_file)

        # Download the ZIP file to the user's local machine
        if zip_file:
            files.download(zip_file)
    else:
        print("Please upload a valid CSV and logo file.")

Please upload your CSV file and logo.
